Jupyter Tutorial: The NetPyNE batchtools subpackage
How to use the `specs` and `comm` to communicate with the `batchtools` `dispatcher`


For each individual `sim`, communication with the `batchtools` `dispatcher` occurs through the `specs` and `comm` objects

In [ ]:
from netpyne.batchtools import specs, comm

the `specs` object is an instantiation of a custom class extending the `batchtk` `Runner` ...

In [ ]:
help(type(specs))

From this `specs` object, we can similarly call `specs.NetParams` and `specs.SimConfig` to create the NetPyNE objects...

In [ ]:
help(specs.NetParams)
help(specs.SimConfig)

The `batchtools` job submission tool uses `environmental variables` to pass values to our `config` object created by `specs.SimConfig`, these `environmental variables` are captured during the `specs` `object creation` which occurs during the batchtools `import` (from the `batchtools` `__init__.py`:
```
from netpyne.batchtools.runners import NetpyneRunner
specs = NetpyneRunner()
```

Let's `export` some `environmental variables` to pass values to our `config` object. When this is handled by the `batchtools` `subpackage`, this occurs automatically...

In [ ]:
%env STRRUNTK0   =foo.bar=baz
%env FLOATRUNTK1 =float_val=7.7
from netpyne.batchtools import NetpyneRunner
specs = NetpyneRunner()

One way of retrieving these values is by calling `specs.get_mappings()`

In [ ]:
print(specs.get_mappings())

Now, let's create our `config` object using the `specs.SimConfig()` constructor
This `config` object will hold a `dictionary` such that the initial values `foo['bar']` = `not_baz` and a `float_val` = `3.3`

In [ ]:
cfg = specs.SimConfig()
cfg.foo = {'bar': 'not_baz', 'qux': 'quux'}
cfg.float_val = 3.3
print("cfg.foo['bar'] = {}".format(cfg.foo['bar']))
print("cfg.float_val = {}".format(cfg.float_val))

Finally, calling the `cfg.update_cfg()` method will overwrite the original values with our environment values, (`baz` and `7.7`)...

in NetPyNE, this was originally handled with the:
```
try:
    from __main__ import cfg
except:
    from cfg import cfg
```
API idiom in the `netParams.py` file...
 
as well as the 
```
cfg, netParams = sim.readCmdLineArgs(simConfigDefault='src/cfg.py', netParamsDefault='src/netParams.py')
```
API idiom in the `init.py` file...

using the `batchtools` subpackage, we can treat the `cfg` as an object and pass it between scripts via `import` statements...
in `netParams.py`...
```
from cfg import cfg
cfg.update()
```
in `init.py`...
```
from netParams import cfg, netParams
sim.createSimulateAnalyze(simConfig=cfg, netParams=netParams)
```

In [ ]:
print("prior to  cfg.update()")
print("cfg.foo['bar'] = {}".format(cfg.foo['bar']))
print("cfg.float_val = {}".format(cfg.float_val))
print()
cfg.update() # call update_cfg to update values in the cfg object with values assigned by batch
print("after the cfg.update()")
print("cfg.foo['bar'] = {}".format(cfg.foo['bar']))
print("cfg.float_val = {}".format(cfg.float_val))

Finally, the `comm object` is used to report to the monitoring `dispatcher object`
the means of communication is dependent on which `dispatcher object` is instantiated, and communicated through environmental variables
in this case, since there is no `dispatcher object` the `comm` methods will simply perform `pass operations`

In [ ]:
comm.initialize() # initializes comm object, establishing channel to communicate with the host dispatcher object

In [ ]:
print(comm.is_host()) # returns a boolean IF the calling process is the 0th ranked  parallelcontext, similar to sim.pc.rank == 0

In [ ]:
comm.send('message') # sends 'message' to the `dispatcher object`

In [ ]:
comm.close() #finalizes communication, closes any resources used to communicate with the `dispatcher object`